<a href="https://colab.research.google.com/github/tashisharma10/credit-card-fraud-detection-/blob/main/Credit_Card_fraud_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [3]:
#loading the dataset into pandas Dataframe
import csv
filename = '/content/creditcard.csv'


def read_csv_with_error_handling(filename):
    data = []
    with open(filename, 'r', newline='', encoding='utf-8') as file:
        reader = csv.reader(file)
        header = next(reader)
        data.append(header)
        for line_number, row in enumerate(reader, start=2):
            try:
                if len(row) != len(header):
                    raise ValueError(f"Unexpected number of fields in line {line_number}")
                data.append(row)
            except ValueError as e:
                print(f"Skipping line {line_number}: {e}")

    # Convert data list to DataFrame
    df = pd.DataFrame(data[1:], columns=data[0])
    return df

try:
    dataset = read_csv_with_error_handling(filename)
    print("CSV file loaded successfully.")
except Exception as e:
    print(f"Error loading CSV file: {e}")

Skipping line 17919: Unexpected number of fields in line 17919
CSV file loaded successfully.


In [4]:
dataset.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.3598071336738,-0.0727811733098497,2.53634673796914,1.37815522427443,-0.338320769942518,0.462387777762292,0.239598554061257,0.0986979012610507,0.363786969611213,...,-0.018306777944153,0.277837575558899,-0.110473910188767,0.0669280749146731,0.128539358273528,-0.189114843888824,0.133558376740387,-0.0210530534538215,149.62,0
1,0,1.19185711131486,0.26615071205963,0.16648011335321,0.448154078460911,0.0600176492822243,-0.0823608088155687,-0.0788029833323113,0.0851016549148104,-0.255425128109186,...,-0.225775248033138,-0.638671952771851,0.101288021253234,-0.339846475529127,0.167170404418143,0.125894532368176,-0.00898309914322813,0.0147241691924927,2.69,0
2,1,-1.35835406159823,-1.34016307473609,1.77320934263119,0.379779593034328,-0.503198133318193,1.80049938079263,0.791460956450422,0.247675786588991,-1.51465432260583,...,0.247998153469754,0.771679401917229,0.909412262347719,-0.689280956490685,-0.327641833735251,-0.139096571514147,-0.0553527940384261,-0.0597518405929204,378.66,0
3,1,-0.966271711572087,-0.185226008082898,1.79299333957872,-0.863291275036453,-0.0103088796030823,1.24720316752486,0.23760893977178,0.377435874652262,-1.38702406270197,...,-0.108300452035545,0.00527359678253453,-0.190320518742841,-1.17557533186321,0.647376034602038,-0.221928844458407,0.0627228487293033,0.0614576285006353,123.5,0
4,2,-1.15823309349523,0.877736754848451,1.548717846511,0.403033933955121,-0.407193377311653,0.0959214624684256,0.592940745385545,-0.270532677192282,0.817739308235294,...,-0.00943069713232919,0.79827849458971,-0.137458079619063,0.141266983824769,-0.206009587619756,0.502292224181569,0.219422229513348,0.215153147499206,69.99,0


In [5]:
dataset.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
17912,29027,1.41287437543809,-0.5252238221197,-0.0645896838294458,-0.917303429797941,-0.575127482619485,-0.460096901202374,-0.492201084695999,-0.0817521906722135,-1.06207673028173,...,0.220813963764349,0.504571498322132,-0.215697266496288,-0.317647507728772,0.678750968968079,-0.0916954832808221,-0.0146166666180345,-0.00554255104370857,20.9,0
17913,29027,-0.422158923901284,0.231117678266983,1.6667112895492,0.45197613522523,-0.203598282149083,0.0972439190079324,-0.0396661513589662,0.354218113260035,0.0624631485805656,...,0.110908512422178,0.435121322466068,-0.0566580335287525,0.265866986631929,-0.548204075595545,0.73401311591403,0.117023257679211,0.130972165835243,9,0
17914,29030,1.17738650143565,-0.215585181206128,0.202971980116228,0.215323470736423,-0.0293123898108017,0.601788020564596,-0.297020796539347,0.188082074452523,0.436369727516284,...,-0.0558418660184012,0.0759026366563869,-0.187119543312012,-0.717797855649579,0.555294267766936,0.73153149162019,-0.0221117897429061,-0.0109285697635781,25,0
17915,29030,-0.553745834594479,0.880858093053387,1.64482109532872,-0.132657221113966,0.120940193994634,-0.267410890950453,0.466891625975303,0.222443324730471,-0.639624110710254,...,-0.133339329575457,-0.348661800753882,0.0299470672179367,0.19996234114131,-0.328384174127291,0.0715108606543779,0.275486690655574,0.110194638947242,0.89,0
17916,29030,-2.84463155959079,3.71795970249877,-7.16542791751608,4.12041906076698,-2.99103894016062,-2.9423261314091,-4.92518687932855,2.20433707003411,-2.66361274090678,...,0.894495433892672,-0.340246328581348,0.0122218395658478,-0.0596789523189639,-0.104337740172113,-0.295883791407814,1.32622778753906,0.322687704849641,89.99,0


In [6]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17917 entries, 0 to 17916
Data columns (total 31 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Time    17917 non-null  object
 1   V1      17917 non-null  object
 2   V2      17917 non-null  object
 3   V3      17917 non-null  object
 4   V4      17917 non-null  object
 5   V5      17917 non-null  object
 6   V6      17917 non-null  object
 7   V7      17917 non-null  object
 8   V8      17917 non-null  object
 9   V9      17917 non-null  object
 10  V10     17917 non-null  object
 11  V11     17917 non-null  object
 12  V12     17917 non-null  object
 13  V13     17917 non-null  object
 14  V14     17917 non-null  object
 15  V15     17917 non-null  object
 16  V16     17917 non-null  object
 17  V17     17917 non-null  object
 18  V18     17917 non-null  object
 19  V19     17917 non-null  object
 20  V20     17917 non-null  object
 21  V21     17917 non-null  object
 22  V22     17917 non-null

In [7]:
#checking the number of missing values in each column
dataset.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [9]:
#distribution of legit transaction and fraudulent transaction
dataset["Class"].value_counts()

Class
0    17836
1       81
Name: count, dtype: int64

This data is highly imbalanced

0-Normal transaction

1-fraudulent transaction

In [11]:
print(dataset['Class'].unique())

['0' '1']


In [12]:
print(dataset['Class'].dtype)


object


In [13]:
dataset['Class'] = dataset['Class'].astype(str).str.strip()



In [14]:
# separating the data for analysis
legit = dataset[dataset['Class'] == '0'].copy()
fraud = dataset[dataset['Class'] == '1'].copy()

In [15]:
print("Shape of legitimate transactions data:", legit.shape)
print("Shape of fraudulent transactions data:", fraud.shape)

Shape of legitimate transactions data: (17836, 31)
Shape of fraudulent transactions data: (81, 31)


In [16]:
# Print data types of all columns
print(dataset.dtypes)

Time      object
V1        object
V2        object
V3        object
V4        object
V5        object
V6        object
V7        object
V8        object
V9        object
V10       object
V11       object
V12       object
V13       object
V14       object
V15       object
V16       object
V17       object
V18       object
V19       object
V20       object
V21       object
V22       object
V23       object
V24       object
V25       object
V26       object
V27       object
V28       object
Amount    object
Class     object
dtype: object


In [17]:
#statistical measures of data
legit.Amount.describe()


count     17836
unique     5668
top           1
freq        793
Name: Amount, dtype: object

In [18]:
fraud.Amount.describe()

count     81
unique    20
top        1
freq      40
Name: Amount, dtype: object

In [19]:


# Convert 'Amount' column to string type explicitly
dataset['Amount'] = dataset['Amount'].astype(str)

# Clean 'Amount' column (remove all non-numeric characters except decimal point)
dataset['Amount'] = dataset['Amount'].str.replace('[^\d.]', '', regex=True)

# Convert 'Amount' column to numeric type with errors='coerce'
dataset['Amount'] = pd.to_numeric(dataset['Amount'], errors='coerce')

# Check for NaN values after conversion
nan_values = dataset['Amount'].isna().sum()
print(f"Number of NaN values after conversion: {nan_values}")

# Drop rows with NaN values in 'Amount' column (if necessary)
dataset = dataset.dropna(subset=['Amount'])

# Verify conversion by printing unique values in 'Amount' column
print(dataset['Amount'].unique())

# Group by 'Class' and calculate mean of 'Amount' column
try:
    mean_by_class = dataset.groupby('Class')['Amount'].mean()
    print(mean_by_class)
except Exception as e:
    print(f"Error calculating mean: {e}")


Number of NaN values after conversion: 0
[149.62   2.69 378.66 ...  53.91 100.8   20.9 ]
Class
0    67.365025
1    98.105926
Name: Amount, dtype: float64


Under-Sampling

Build a sample dataset containing similar distribution of normal transactions and Fraudulent Transactions

Number of Fraudulent Transactions - 492

In [21]:
legit_sample = legit.sample(n=81)

Concatenating two DataFrames

In [22]:
new_dataset = pd.concat([legit_sample, fraud], axis=0)

In [23]:
new_dataset.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
16988,28344,-1.80429505777672,-1.4505844173101,2.33454571887993,0.834795946506709,0.616952623706557,-0.68587256373176,-0.762725022266028,0.32295272698567,-0.19785256130238,...,0.459016652580389,0.761161380682515,0.378015218803541,0.39606918286437,-0.115425272405466,0.611738866644491,0.0124897473266889,0.1569939596587,136,0
15317,26703,-5.81395361296379,2.32093522455802,-4.19638323702896,1.2596486685943,-2.81049333015377,-0.811380181354562,-1.58510258780386,3.34051070114098,-0.358766602375756,...,0.0996134241547109,0.340697058390563,0.0984234561784513,0.0397999022975043,-0.389200050434667,-0.364226435392853,0.322639079940741,-0.0674010448730054,99.99,0
16144,27561,-0.341824899757957,1.08223862429594,1.28255329323865,0.0692507022265473,-0.0262609491582499,-1.00530610570533,0.658144879010942,-0.0335162089158447,-0.290153976803487,...,-0.272196939700784,-0.745530094109284,-0.00924601595711153,0.315592246267203,-0.184390372248564,0.0742039496099822,0.241528446745599,0.0967857602501465,1.78,0
17256,28573,1.09128950712519,0.213173398577482,1.65848924392997,2.97995537521098,-0.889280755172377,0.226514651192741,-0.531010210793211,0.124608754640345,0.280928428811294,...,0.0540222727323337,0.566891372685492,-0.100557972998018,0.795781463354948,0.608824600130598,0.189273353164346,0.0559226519535408,0.0324865230575377,0,0
9819,14506,1.1858444499518,-0.0749032379270707,0.73560857420579,0.0775300855835321,-0.389386817991129,0.158510819333144,-0.599101400625403,0.0824838660540786,1.43584668983733,...,-0.00487971960848248,0.267878897808675,-0.0423798266765011,-0.267814953068815,0.16172623405845,1.0620720202739,-0.0744719480236534,-0.0103169983712773,14.95,0


In [24]:
new_dataset.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
17317,28625,-27.84818067198,15.5981926625554,-28.923755945104,6.41844174657532,-20.346228155413,-4.82820246578157,-19.2108964179086,18.329405525597,-3.66873493861418,...,1.8021491336386,-2.06293427611466,-1.26984343615924,0.165409188492223,1.99949911253199,-0.211058522431997,1.32480949279508,0.388090177732107,99.99,1
17366,28658,-28.5242675938406,15.8769229879536,-29.4687320925264,6.44759140152748,-20.7860000418837,-4.86561341755669,-19.5010840750712,18.7488719520883,-3.64298981925263,...,1.80576978392608,-2.11937616760819,-1.31744962200248,0.169845630469178,2.05168733506275,-0.210502000459411,1.30173396497076,0.380246181418509,99.99,1
17407,28692,-29.2003285905744,16.1557014298057,-30.0137124857248,6.47673117996833,-21.2258096535165,-4.90299739658728,-19.791248405247,19.1683273897301,-3.61724178604255,...,1.8093709332884,-2.17581520342142,-1.3651041075509,0.174286359566544,2.10386807204715,-0.209943999130567,1.27868097084218,0.372392714338541,99.99,1
17453,28726,-29.8763655139763,16.4345245512223,-30.5586968214292,6.50586178736296,-21.6656542956569,-4.94035632949796,-20.0813910746494,19.5877726234404,-3.59149104711596,...,1.81295397464878,-2.23225158753151,-1.41280344282556,0.178731069561437,2.15604185816706,-0.20938462411377,1.25564890085966,0.364530452975509,99.99,1
17480,28755,-30.552380043581,16.7133892350242,-31.1036848245812,6.53498386355181,-22.105531524316,-4.97769196369245,-20.371513594495,20.0072083651213,-3.56573779124378,...,1.8165201713562,-2.28868550494101,-1.46054449877578,0.183179482716819,2.20820917836653,-0.208823971247713,1.23263629519904,0.356660010884047,99.99,1


In [25]:
new_dataset['Class'].value_counts()

Class
0    81
1    81
Name: count, dtype: int64

In [26]:
# Convert 'Amount' column to string type explicitly
new_dataset['Amount'] = new_dataset['Amount'].astype(str)

# Clean 'Amount' column (remove all non-numeric characters except decimal point)
new_dataset['Amount'] = new_dataset['Amount'].str.replace('[^\d.]', '', regex=True)

# Convert 'Amount' column to numeric type with errors='coerce'
new_dataset['Amount'] = pd.to_numeric(new_dataset['Amount'], errors='coerce')

# Check for NaN values after conversion
nan_values = new_dataset['Amount'].isna().sum()
print(f"Number of NaN values after conversion: {nan_values}")

# Drop rows with NaN values in 'Amount' column (if necessary)
new_dataset = new_dataset.dropna(subset=['Amount'])

# Verify conversion by printing unique values in 'Amount' column
print(new_dataset['Amount'].unique())

# Group by 'Class' and calculate mean of 'Amount' column
try:
    mean_by_class = new_dataset.groupby('Class')['Amount'].mean()
    print(mean_by_class)
except Exception as e:
    print(f"Error calculating mean: {e}")


Number of NaN values after conversion: 0
[1.36000e+02 9.99900e+01 1.78000e+00 0.00000e+00 1.49500e+01 1.59500e+01
 6.00000e+01 2.31000e+01 3.71300e+01 9.99000e+00 8.90000e-01 4.50000e+01
 1.49000e+01 2.10000e+01 1.00000e-02 5.84000e+00 1.20500e+01 2.13910e+02
 2.15500e+01 1.98000e+00 1.14890e+02 5.00000e+00 5.26000e+00 2.57280e+02
 7.61000e+00 1.52950e+02 1.88000e+00 3.04000e+00 1.68410e+02 3.76000e+00
 4.83200e+01 8.92000e+00 3.07500e+01 9.91600e+01 9.90000e+00 4.32900e+01
 3.90000e+01 2.02200e+01 5.33000e+01 1.60700e+01 1.00000e+00 4.49000e+00
 9.00000e+01 4.79000e+01 7.42200e+01 3.30000e+01 1.00000e+01 1.49700e+01
 1.35860e+02 2.37090e+02 4.14000e+00 1.74300e+01 7.66000e+01 3.00000e+01
 7.00000e+00 1.18500e+01 8.99000e+00 9.90000e-01 1.99900e+01 4.95000e+00
 8.22900e+01 1.49900e+01 1.53000e+02 1.84220e+02 4.30680e+02 1.59900e+01
 1.77150e+02 1.89600e+01 5.75800e+01 1.50800e+01 2.00000e+00 2.10300e+01
 2.56710e+02 5.29000e+02 2.39930e+02 5.90000e+01 1.10000e+00 1.80968e+03
 3.03900e+

Splitting the data into Features & Targets

In [27]:
X = new_dataset.drop(columns='Class', axis=1)
Y = new_dataset['Class']

In [28]:
print(X)

        Time                  V1                   V2                 V3  \
16988  28344   -1.80429505777672     -1.4505844173101   2.33454571887993   
15317  26703   -5.81395361296379     2.32093522455802  -4.19638323702896   
16144  27561  -0.341824899757957     1.08223862429594   1.28255329323865   
17256  28573    1.09128950712519    0.213173398577482   1.65848924392997   
9819   14506     1.1858444499518  -0.0749032379270707   0.73560857420579   
...      ...                 ...                  ...                ...   
17317  28625     -27.84818067198     15.5981926625554   -28.923755945104   
17366  28658   -28.5242675938406     15.8769229879536  -29.4687320925264   
17407  28692   -29.2003285905744     16.1557014298057  -30.0137124857248   
17453  28726   -29.8763655139763     16.4345245512223  -30.5586968214292   
17480  28755    -30.552380043581     16.7133892350242  -31.1036848245812   

                       V4                   V5                  V6  \
16988   0.8347959

In [29]:
print(Y)

16988    0
15317    0
16144    0
17256    0
9819     0
        ..
17317    1
17366    1
17407    1
17453    1
17480    1
Name: Class, Length: 162, dtype: object


In [46]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=1)

In [47]:
print(X.shape, X_train.shape, X_test.shape)

(162, 30) (129, 30) (33, 30)


Model Training

Logistic Regression

In [48]:
model = LogisticRegression(max_iter=1000)
#training the Logistic Regression model with training data
model.fit(X_train, Y_train)
print("Logistic Regression Model Parameters:")

print(f"Intercept: {model.intercept_[0]}")
print(f"C: {model.C}")
print(f"Class weight: {model.class_weight}")
print(f"Dual: {model.dual}")
print(f"Fit intercept: {model.fit_intercept}")
print(f"Intercept scaling: {model.intercept_scaling}")
print(f"L1 ratio: {model.l1_ratio}")
print(f"Maximum iterations: {model.max_iter}")
print(f"Multi class: {model.multi_class}")
print(f"N jobs: {model.n_jobs}")
print(f"Penalty: {model.penalty}")
print(f"Random state: {model.random_state}")
print(f"Solver: {model.solver}")
print(f"Tolerance: {model.tol}")
print(f"Verbose: {model.verbose}")
print(f"Warm start: {model.warm_start}")

Logistic Regression Model Parameters:
Intercept: -4.941040499109934
C: 1.0
Class weight: None
Dual: False
Fit intercept: True
Intercept scaling: 1
L1 ratio: None
Maximum iterations: 1000
Multi class: auto
N jobs: None
Penalty: l2
Random state: None
Solver: lbfgs
Tolerance: 0.0001
Verbose: 0
Warm start: False


Model Evaluation

Accuracy Score

In [49]:
# accuracy on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [50]:
print('Accuracy on Training data : ', training_data_accuracy)

Accuracy on Training data :  1.0


In [51]:
# accuracy on test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [52]:
print('Accuracy score on Test Data : ', test_data_accuracy)


Accuracy score on Test Data :  1.0
